## Yield calculator.

This notebook contains the code to calculate yields from e.g. Battino et a. (2019, 2021) or Ritter et al. (2018) MESA models + stellar postprocessing.

The yields we use here are the integrated total ejected mass yield, i.e.

$$
EM = \int X_{i, \rm surf}(t)\,\dot{M}(t)\ dt
$$

To calculate this integral, we simply sum over the surface abundance (times the change in mass) for each timestep of the star. We additionally add the mass contained outside the final reminant at the final snapshot time as this should be included as well.

In [ ]:
from nugridpy import mesa as ms
from nugridpy import nugridse as mp

from datetime import datetime
from dataclasses import dataclass

import numpy as np
import re

In [ ]:
@dataclass 
class StarIsotopeMasses:
    """ A class to store surface isotope data and basic star properties necessary to calculate wind yields. """
    mass_initial : float
    metallicity : float
    modelname : str
    
    iso_massf : list
    mass : list
    mass_final_h_free : float
    isotopes : list
    
    @classmethod
    def from_mesa(cls, star_log_path, se_surf_path, *, mass_initial=None, metallicity = None, modelname = ""):
        print("opening files")
        pt_surf = mp.se(
            sedir=se_surf_path
        )
        sl = ms.history_data(
            star_log_path
        )

        isotopes = pt_surf.se.isotopes
    
        print()
        iso_massf = pt_surf.get("iso_massf")
        print()
        mass = pt_surf.get("mass")

        mass_final_h_free = sl.get("h1_boundary_mass")[-1]
        if mass_initial is None:
            mass_initial = mass[0]
        if metallicity is None:
            metallicity = -1 # TODO actually calculate
        
        return cls(
            mass_initial = mass_initial,
            metallicity = metallicity,
            modelname = modelname, 
            iso_massf = iso_massf,
            mass = mass,
            mass_final_h_free = mass_final_h_free,
            isotopes = isotopes,
        )
    
    

In [ ]:
def calc_wind_yields_postmesa(starisos):
    Xf = starisos.iso_massf[-1]

    yields = (starisos.mass[-1] - starisos.mass_final_h_free) * Xf

    return yields

In [ ]:
def calc_wind_yields_mesaevo(mass, iso_massf):
    EM = np.zeros(len(iso_massf[0]))
    dm = -np.diff(mass)

    for i in range(len(mass) - 1):
        if i % 1000 == 0:
            print(f"summing {i} / {len(mass)}", end="\r")
            
        X = iso_massf[i + 1]
        EM += dm[i] * X

    return EM

In [ ]:
def write_datatable(filename, data: dict):
    columns = list(data.keys())
    
    with open(filename, "a") as file:
        # write header
        
        for col in columns:
            print(f"{col:15} ", file=file, end="")
        print(file=file)

        Nrows = len(data[columns[0]])
        
        for i in range(Nrows):
            for col in columns:
                val = data[col][i]
                T = type(val)
                if T is str:
                    print(f"{val:15} ", file=file, end="")
                elif isinstance(val, (int)):
                    print(f"{val:15i} ", file=file, end="")
                elif isinstance(val, (float, np.floating)):
                    print(f"{val:15.6e} ", file=file, end="")
                else:
                    raise Exception(f"dtype not known: {type(val)}")
                
            print(file=file)
            # end col loop
            
        # end row loop
    # end open file
    

In [ ]:
@dataclass
class CalculatedYields:
    modelname : str
    mass_initial : float
    metallicity : float
    
    mass_final_mesa : float
    mass_final_h_free : float
    
    isotopes : list
    initial_abundances : list
    wind_yields : list
    wind_yields_mesaevo : list
    wind_yields_postmesa : list
    
    
    @classmethod 
    def from_starisos(cls, starisos):
        initial_abundances = starisos.iso_massf[0]
        initial_mass = starisos.mass_initial
        remnant_mass = starisos.mass_final_h_free
        
        wind_yields_mesaevo = calc_wind_yields_mesaevo(starisos.mass, starisos.iso_massf)
        wind_yields_postmesa = calc_wind_yields_postmesa(starisos)
        wind_yields = wind_yields_mesaevo + wind_yields_postmesa
        
        return cls(
            modelname = starisos.modelname,
            mass_initial = starisos.mass_initial,
            metallicity = starisos.metallicity,
            mass_final_h_free = starisos.mass_final_h_free,
            mass_final_mesa = starisos.mass[-1],
            isotopes = starisos.isotopes,
            initial_abundances = initial_abundances,
            wind_yields = wind_yields,
            wind_yields_postmesa = wind_yields_postmesa,
            wind_yields_mesaevo = wind_yields_mesaevo
        )
           
        
    def _write_header(self, filename):
        with open(filename, "w") as file:
            print("# Battino yields", file=file)
            print(f"# Model: {self.modelname}", file=file)
            print(f"# Mini: {self.mass_initial:0.2f}", file=file)
            print(f"# Z: {self.metallicity:0.2e}", file=file)
            print(f"# Mfinal: {self.mass_final_h_free}", file=file)
            print(f"# Mfinal (MESA): {self.mass_final_mesa}", file=file)
            print(
                f"# Created by Daniel Boyea on {datetime.today().strftime('%Y-%m-%d')}",
                file=file,
            )
        
    @property
    def _datadict(self):
        return {
            "isotope": self.isotopes,
            "wind_yield": self.wind_yields,
            "initial_abundance": self.initial_abundances,
            "wind_yield_mesaevo": self.wind_yields_mesaevo,
            "wind_yield_postmesa": self.wind_yields_postmesa,
        }
    
    
    def write(self, filename=None):
        if filename is None:
            filename = f"yields_{self.modelname}.txt"
            
        self._write_header(filename)
        write_datatable(filename, self._datadict)
        

# Ritter 2018

In [ ]:
def calc_all_ritter18(modelname):
    sl = get_sl_path_ritter18(modelname)
    se = get_se_path_ritter18(modelname) + "/H5_surf"
    
    mass_initial, metallicity = get_initial_mass_metallicity(modelname)
    
    
    starisos = StarIsotopeMasses.from_mesa(sl, se, mass_initial=mass_initial, metallicity=metallicity, modelname=modelname)
    yields = CalculatedYields.from_starisos(starisos)
    yields.write()


In [ ]:
def get_se_path_ritter18(modelname):
    setname = get_setname(modelname)
    path = f"/data/nugrid_data/set1ext/{setname}/ppd_wind/{modelname}/"

    return path

In [ ]:
def get_sl_path_ritter18(modelname):
    setname = get_setname(modelname)

    path = f"/data/nugrid_data/set1ext/{setname}/see_wind/{modelname}/LOGS"     
    
    return path

In [ ]:
def get_setname(modelname):
    M, Z = get_initial_mass_metallicity(modelname)
    if Z == 0.02:
        setname = "set1.2"
    elif Z == 0.01:
        setname = "set1.1"
    elif Z == 6.0e-03:
        setname = "set1.3a"
    elif Z == 1.0e-03:
        setname = "set1.4a"
    elif Z == 1.0e-04:
        setname = "set1.5a"
    else:
        raise Exception("set not found")
    return setname

In [ ]:
def get_initial_mass_metallicity(modelname):
    """
    Extracts two float numbers from a string of the format "M<number>Z<number>".
    
    Parameters:
        input_string (str): The input string to parse.
    
    Returns:
        tuple: A tuple containing two floats (M value, Z value).
    """
    pattern = r"M([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)Z([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)"
    match = re.match(pattern, modelname)
    if match:
        m_value = float(match.group(1))
        z_value = float(match.group(2))
        return m_value, z_value
    else:
        raise ValueError(f"String format invalid: {modelname}")

In [ ]:
calc_all_ritter18("M2.00Z2.0e-02");

In [ ]:
calc_all_ritter18("M7.000Z0.0010");

# Battino 2019

In [ ]:
def get_set_battino19(modelname):
    if "z2m2" in modelname:
        setname = "set1.2"
    elif "z3m2" in modelname:
        setname = "set1.3"
    elif "z1m2" in modelname:
        setname = "set1.1"
    else:
        raise Exception("set not found")
        
    return setname

In [ ]:
def get_se_path_battino19(modelname):
    setname = get_set_battino19(modelname)

    path = f"/data/nugrid_data/set1upd/{setname}/ppd_wind/RUN_set1upd_{modelname}/"

    return path

In [ ]:
def get_sl_path_battino19(modelname):
    setname = get_set_battino19(modelname)

    path = f"/data/nugrid_data/set1upd/{setname}/see_wind/MESA_directories/{modelname}/LOGS"   
        
    return path

In [ ]:
def calc_all_battino19(modelname):
    sl = get_sl_path_battino19(modelname)
    se = get_se_path_battino19(modelname) + "/H5_surf"
    
    if "m2z" in modelname:
        mass_initial = 2
    elif "m3z" in modelname:
        mass_initial = 3
    else:
        pass
    
    if "z2m2" in modelname:
        metallicity = 2e-2
    elif "z1m2" in modelname:
        metallicity = 1e-2
    
    
    starisos = StarIsotopeMasses.from_mesa(sl, se, mass_initial=mass_initial, metallicity=metallicity, modelname=modelname)
    yields = CalculatedYields.from_starisos(starisos)
    yields.write()


In [ ]:
calc_all_battino19("m2z2m2")

In [ ]:
calc_all_battino19("m3z2m2")

In [ ]:
calc_all_battino19("m2z1m2")

In [ ]:
calc_all_battino19("m3z1m2")

# Battino et al. 2021

because the nugrid's star log seems to require writing the .logsa file, I have copied the directories into this folder for the models in this set so that ms.history_data does not fail.

In [ ]:
def get_se_path_battino(modelname):
    setname = get_set(modelname)
    if "set1.0" in setname:
        path = f"/data/nugrid_data/set1upd/{setname}/ppd_wind/{modelname}/"
    else:
        path = f"/data/nugrid_data/set1upd/{setname}/ppd_wind/RUN_set1upd_{modelname}/"

    return path

In [ ]:
def get_sl_path_battino(modelname):
    setname = get_set(modelname)
    if "set1.0" in setname: 
        path = f"/data/nugrid_data/set1upd/{setname}/see_wind/{modelname}/LOGS"
    else:
        path = f"/data/nugrid_data/set1upd/{setname}/see_wind/MESA_directories/{modelname}/LOGS"   
        
    return path

In [ ]:
def calc_all_battino21(modelname, **kwargs):
    starisos = StarIsotopeMasses.from_mesa(f"{modelname}/LOGS", f"{modelname}/H5_surf", modelname=modelname, **kwargs)
    yields = CalculatedYields.from_starisos(starisos)
    yields.write()
    return yields

In [ ]:
calc_all_battino21("m3z1m3", mass_initial=3, metallicity=1e-3)

In [ ]:
calc_all_battino21("m2z1m3", mass_initial=2, metallicity=1e-3);

In [ ]:
calc_all_battino21("m2z2m3", mass_initial=2, metallicity=2e-3);

In [ ]:
calc_all("m3z1m3-bigpoc")

In [ ]:
calc_all_lowz("m2z1m3")

In [ ]:
calc_all_lowz("m3z1m3")

In [ ]:
calc_all_lowz("m2z2m3")